# ANLI Baseline with LLM

You have to implement in this notebook a baseline for ANLI classification using an LLM.
This baseline must be implemented using DSPy.



In [26]:
# Load API key from file
import configparser
import os

# Read the key from grok_key.ini
with open('grok_key.ini', 'r') as f:
    line = f.read().strip()
    # Extract the key from "export XAI_API_KEY=your_key_here"
    if line.startswith('export XAI_API_KEY='):
        api_key = line.split('=', 1)[1]
        os.environ['XAI_API_KEY'] = api_key
        print("API key loaded successfully")
    else:
        print("Could not parse API key from file")

API key loaded successfully


In [27]:
# Configure the DSPy environment with the language model - for grok the parameters must be:
# env variable should be in os.environ['XAI_API_KEY']
# "xai/grok-3-mini"
import os
import dspy

#xai
lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
dspy.configure(lm=lm)
# for ollama
# lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434', api_key='')
# dspy.configure(lm=lm)
# lm = dspy.LM(
#     "ollama/llama3.2:latest",
#     api_base="http://localhost:11434",
#     format="json"        # litellm translates this to Ollama's stream=false
# )
#dspy.configure(lm=lm, adapter=dspy.JSONAdapter())  # ask DSPy to keep JSON

In [ ]:
from typing import Literal

# Implement the DSPy classifier 
class NLIClassifier(dspy.Signature):
    """Analyze the logical relationship between a premise and hypothesis."""
    premise: str = dspy.InputField(desc="A foundational statement or passage.")
    hypothesis: str = dspy.InputField(desc="A claim to evaluate against the premise.")
    reasoning: str = dspy.OutputField(prefix="Reasoning: Let's think step by step in order to")
    label: Literal["entailment", "neutral", "contradiction"] = dspy.OutputField()

class DSPyNLI(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classifier = dspy.ChainOfThought(NLIClassifier)
    
    def forward(self, premise, hypothesis):
        return self.classifier(premise=premise, hypothesis=hypothesis)

## Load ANLI dataset

In [29]:
from datasets import load_dataset

dataset = load_dataset("facebook/anli")
dataset = dataset.filter(lambda x: x['reason'] != None and x['reason'] != "")

In [30]:
dataset

DatasetDict({
    train_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 2923
    })
    dev_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 4861
    })
    dev_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 13375
    })
    dev_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1200


In [ ]:
import random
from sklearn.metrics import f1_score  
import dspy
from dspy.teleprompt import MIPROv2


random.seed(42)

# Random sampling for trainset (to avoid order bias)
dev_r3_list = dataset['dev_r3'].to_list()
sampled_examples = random.sample(dev_r3_list, k=min(100, len(dev_r3_list)))  

# Convert to DSPy Examples
def convert_dict(ex):
    return dspy.Example(
        premise=ex["premise"],
        hypothesis=ex["hypothesis"],
        label={0: "entailment", 1: "neutral", 2: "contradiction"}[ex["label"]]
    ).with_inputs("premise", "hypothesis")

trainset = [convert_dict(x) for x in sampled_examples]

# F1 metric
def f1_metric(example, pred, trace=None):
    gold = example.label.strip().lower()
    predicted = pred.label.strip().lower() if hasattr(pred, 'label') else 'neutral'  # Fallback for errors
    labels = ['entailment', 'neutral', 'contradiction']
    return f1_score([gold], [predicted], labels=labels, average='macro', zero_division=0)  


# Instantiate your DSPy model
dspy_model = DSPyNLI()

# MIPROv2 optimizer 
miprov2 = MIPROv2(
    metric=f1_metric,  # Use F1 for optimization 
    verbose=True,
    auto=None,  # Disable auto mode to set custom params
    num_candidates=12,  # Required when auto=None; controls candidates for few-shots/instructions
    init_temperature=1.0  
)

# Compile the optimized classifier
compiled_clf = miprov2.compile(
    dspy_model,
    trainset=trainset,
    num_trials=15,  # Number of optimization trials
    max_bootstrapped_demos=8,  # Demos per few-shot set
    max_labeled_demos=4,
    minibatch=True,  # Enable minibatching for efficiency
    minibatch_size=20,  
    minibatch_full_eval_steps=5,  # Full val eval every 5 minibatch steps
    requires_permission_to_run=False  # Skip confirmation prompts
)

2025/07/21 19:52:26 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/21 19:52:26 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.



2025/07/21 19:52:26 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


 50%|█████     | 10/20 [00:00<00:00, 556.27it/s]


Bootstrapped 8 full traces after 10 examples for up to 1 rounds, amounting to 10 attempts.
Bootstrapping set 4/12


...

Predictor 0
i: You are an expert in Natural Language Inference (NLI). Your task is to meticulously evaluate the relationship between a provided premise—a foundational statement—and a hypothesis—a claim to be assessed. Begin by generating a clear, step-by-step reasoning process that explores the implications, potential ambiguities, and logical connections or contradictions between the two. Draw on general knowledge where relevant, but base your analysis strictly on the information in the premise. After your reasoning, output the final label as one of the following: 'entailment' (if the hypothesis logically follows from the premise), 'contradiction' (if the hypothesis opposes the premise), or 'neutral' (if there is no clear relationship). Format your response with "Reasoning: [your step-by-step explanation]" followed by "Label: [your chosen label]".
p: Label:


Average Metric: 4.67 / 20 (23.3%): 100%|██████████| 20/20 [00:00<00:00, 3412.50it/s]

2025/07/21 19:52:28 INFO dspy.evaluate.evaluate: Average Metric: 4.666666666666666 / 20 (23.3%)
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 23.33 on minibatch of size 20 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 11'].
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [18.33, 20.0, 16.67, 16.67, 21.67, 21.67, 18.33, 23.33, 23.33, 21.67, 18.33, 16.67, 23.33, 28.33, 23.33]
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [20.42, 20.83, 21.25]
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 21.25
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 19 / 19 - Full Evaluation =====
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 28.33)


Average Metric: 17.33 / 80 (21.7%): 100%|██████████| 80/80 [00:00<00:00, 1166.23it/s]

2025/07/21 19:52:28 INFO dspy.evaluate.evaluate: Average Metric: 17.33333333333334 / 80 (21.7%)


2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 21.67
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [20.42, 20.83, 21.25, 21.67]
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 21.67
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/21 19:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 21.67!


## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [33]:
from evaluate import load

accuracy = load("accuracy")
precision = load("precision")
recall = load("recall")
f1 = load("f1")


In [34]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [35]:
clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])

{'accuracy': 0.6666666666666666,
 'f1': 0.6666666666666666,
 'precision': 1.0,
 'recall': 0.5}

## Your Turn

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]

In [37]:
# DeBERTa Setup (for comparison)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
deberta_tokenizer = AutoTokenizer.from_pretrained(model_name)
deberta_model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

def evaluate_deberta(premise, hypothesis):
    inputs = deberta_tokenizer(premise, hypothesis, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = deberta_model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1)[0].cpu().tolist()
    return {"entailment": probs[0], "neutral": probs[1], "contradiction": probs[2]}

def get_deberta_prediction(pred_dict):
    return max(pred_dict, key=pred_dict.get)

In [ ]:
#Evaluation on test_r3 and Comparison to Baseline 
import json
from tqdm import tqdm
from sklearn.metrics import classification_report, cohen_kappa_score
import evaluate  
import pickle  
import pandas as pd  

label_to_idx = {'entailment': 0, 'neutral': 1, 'contradiction': 2}

def compute_agreement_metrics(llm_preds, deberta_preds, gold_labels):
    both_correct = llm_right_deb_wrong = deb_right_llm_wrong = both_wrong = 0
    for lp, dp, g in zip(llm_preds, deberta_preds, gold_labels):
        if lp == g and dp == g:
            both_correct += 1
        elif lp == g and dp != g:
            llm_right_deb_wrong += 1
        elif lp != g and dp == g:
            deb_right_llm_wrong += 1
        else:
            both_wrong += 1
    return {
        'both_correct': both_correct,
        'llm_right_deberta_wrong': llm_right_deb_wrong,
        'deberta_right_llm_wrong': deb_right_llm_wrong,
        'both_incorrect': both_wrong
    }

def convert_dict_for_eval(ex):
    return {
        'premise': ex["premise"],
        'hypothesis': ex["hypothesis"],
        'label': {0: "entailment", 1: "neutral", 2: "contradiction"}[ex["label"]]
    }

test_r3_data = [convert_dict_for_eval(x) for x in dataset['test_r3'].to_list()]
gold_labels = [ex['label'] for ex in test_r3_data]
total_samples = len(test_r3_data)

# LLM Predictions
llm_preds = []
for ex in tqdm(test_r3_data, desc="LLM test_r3"):
    try:
        pred = compiled_clf(premise=ex['premise'], hypothesis=ex['hypothesis']).label.strip().lower()
    except:
        pred = "neutral"  # Fallback for errors
    llm_preds.append(pred)

# DeBERTa Predictions
deberta_preds = []
for ex in tqdm(test_r3_data, desc="DeBERTa test_r3"):
    scores = evaluate_deberta(ex['premise'], ex['hypothesis'])
    pred = get_deberta_prediction(scores)
    deberta_preds.append(pred)

# Prepare int labels
llm_idx_preds = [label_to_idx.get(p, 1) for p in llm_preds]  # Default to neutral (1) if invalid
deberta_idx_preds = [label_to_idx.get(p, 1) for p in deberta_preds]
gold_idx = [label_to_idx[g] for g in gold_labels]

# Compute individual metrics 
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

llm_accuracy = accuracy_metric.compute(predictions=llm_idx_preds, references=gold_idx)['accuracy']
llm_precision = precision_metric.compute(predictions=llm_idx_preds, references=gold_idx, average='macro')['precision']
llm_recall = recall_metric.compute(predictions=llm_idx_preds, references=gold_idx, average='macro')['recall']
llm_f1 = f1_metric.compute(predictions=llm_idx_preds, references=gold_idx, average='macro')['f1']

llm_metrics = {
    'accuracy': llm_accuracy,
    'precision': llm_precision,
    'recall': llm_recall,
    'f1': llm_f1
}

# Four-Way Agreement
agreement = compute_agreement_metrics(llm_preds, deberta_preds, gold_labels)

# Cohen's Kappa
kappa = cohen_kappa_score(llm_idx_preds, deberta_idx_preds)

# Output
print("\nLLM Metrics on test_r3 sample:", llm_metrics)
print("Cohen's Kappa with DeBERTa:", kappa)
print("Four-Way Agreement:", agreement)
print(classification_report(gold_labels, llm_preds))



LLM test_r3:   0%|          | 0/1200 [00:00<?, ?it/s]

DeBERTa test_r3: 100%|██████████| 1200/1200 [01:27<00:00, 13.69it/s]



LLM Metrics on test_r3 sample: {'accuracy': 0.7108333333333333, 'precision': 0.7555692351826728, 'recall': 0.7106135986733001, 'f1': 0.7160884704798122}
Cohen's Kappa with DeBERTa: 0.26798116576541087
Four-Way Agreement: {'both_correct': 459, 'llm_right_deberta_wrong': 394, 'deberta_right_llm_wrong': 135, 'both_incorrect': 212}
               precision    recall  f1-score   support

contradiction       0.80      0.67      0.73       396
   entailment       0.90      0.64      0.75       402
      neutral       0.57      0.82      0.67       402

     accuracy                           0.71      1200
    macro avg       0.76      0.71      0.72      1200
 weighted avg       0.76      0.71      0.72      1200



In [ ]:
# Saveing the results
# Save predictions as pickle files
with open('llm_preds.pkl', 'wb') as f:
    pickle.dump(llm_preds, f)
with open('deberta_preds.pkl', 'wb') as f:
    pickle.dump(deberta_preds, f)
with open('gold_labels.pkl', 'wb') as f:
    pickle.dump(gold_labels, f)

# Save metrics and agreement as JSON
with open('llm_metrics.json', 'w') as f:
    json.dump(llm_metrics, f, indent=4)
with open('agreement.json', 'w') as f:
    json.dump(agreement, f, indent=4)
with open('kappa.json', 'w') as f:
    json.dump({'kappa': kappa}, f, indent=4)

# Save classification report as JSON 
report = classification_report(gold_labels, llm_preds, output_dict=True)
with open('classification_report.json', 'w') as f:
    json.dump(report, f, indent=4)

try:
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv('classification_report.csv')
    print("Saved classification report as CSV (pandas used).")
except NameError:
    print("Pandas not available; skipped CSV export for report.")

print("All results saved successfully!")

Saved classification report as CSV (pandas used).
All results saved successfully!


In [ ]:
#  Results summary
from sklearn.metrics import confusion_matrix

# Overall Metrics Table summary
metrics_data = {
    'Metric': ['Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1', "Cohen's Kappa"],
    'Value': [llm_metrics['accuracy'], llm_metrics['precision'], llm_metrics['recall'], llm_metrics['f1'], kappa]
}

try:
    metrics_df = pd.DataFrame(metrics_data)
    print("Overall LLM Metrics and Agreement:")
    display(metrics_df.style.format({'Value': '{:.3f}'}).set_caption("LLM Performance Summary")) 
except NameError:  # Fallback if pandas not available
    print("Overall LLM Metrics and Agreement (Text Version):")
    for metric, value in zip(metrics_data['Metric'], metrics_data['Value']):
        print(f"{metric}: {value:.3f}")

# Four-Way Agreement Table 
agreement_with_perc = {k: f"{v} ({v / total_samples * 100:.1f}%)" for k, v in agreement.items()}
print("\nFour-Way Agreement Breakdown (Text Version):")
for category, value in agreement_with_perc.items():
    print(f"{category}: {value}")

# Confusion Matrix 
classes = ['entailment', 'neutral', 'contradiction']
cm = confusion_matrix(gold_labels, llm_preds, labels=classes)
print("\nLLM Confusion Matrix on test_r3 (Text Version):")
print("Rows: True Labels | Columns: Predicted Labels")
print("          " + " ".join(f"{c:12}" for c in classes))
for i, row in enumerate(cm):
    print(f"{classes[i]:10}" + " ".join(f"{val:12}" for val in row))

#  Classification Report 
print("\nPer-Class Classification Report (Text Version):")
print(classification_report(gold_labels, llm_preds))

# Export to CSV (if pandas available)
try:
    metrics_df.to_csv('llm_metrics.csv', index=False)
    agreement_df = pd.DataFrame(list(agreement_with_perc.items()), columns=['Category', 'Count (%)'])
    agreement_df.to_csv('agreement_breakdown.csv', index=False)
    report = classification_report(gold_labels, llm_preds, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv('classification_report.csv')
    print("Exported results to CSV files.")
except NameError:
    print("CSV export skipped (pandas not available).")

Overall LLM Metrics and Agreement:


,Metric,Value
0,Accuracy,0.711
1,Macro Precision,0.756
2,Macro Recall,0.711
3,Macro F1,0.716
4,Cohen's Kappa,0.268



Four-Way Agreement Breakdown (Text Version):
both_correct: 459 (38.2%)
llm_right_deberta_wrong: 394 (32.8%)
deberta_right_llm_wrong: 135 (11.2%)
both_incorrect: 212 (17.7%)

LLM Confusion Matrix on test_r3 (Text Version):
Rows: True Labels | Columns: Predicted Labels
          entailment   neutral      contradiction
entailment         258          127           17
neutral             22          331           49
contradiction           7          125          264

Per-Class Classification Report (Text Version):
               precision    recall  f1-score   support

contradiction       0.80      0.67      0.73       396
   entailment       0.90      0.64      0.75       402
      neutral       0.57      0.82      0.67       402

     accuracy                           0.71      1200
    macro avg       0.76      0.71      0.72      1200
 weighted avg       0.76      0.71      0.72      1200

Exported results to CSV files.
